In [24]:
from astropy.constants import M_sun, R_sun, G, sigma_sb, k_B
from math import pi
from astropy import units
M = 3.285*(10**23)*units.kg
Q = 3.846*(10**26)*units.W
R = units.AU

Calculating the sphere's temperature
$$
\dot{Q} = \sigma A T^4
$$

For computational limit

$$
E >= k_B T \cdot \ln{2}
$$


In [29]:
T = ((Q/(sigma_sb*4*pi*R**2)).decompose())**(1./4)
T

<Quantity 394.07983603 K>

That comes out to 121 C, which is... hot, but when you look, actually correct. I had to Google that to reassure myself.
https://canmom.art/physics/dyson-spheres#:~:text=So%20you%20have%20T%20%E2%88%9D,is%20about%20120%20%E2%88%98%20C%20.
Onto the intent of this sheet, we can get a maximum nuber of computations per second with a Dyson sphere.

In [32]:
(Q/(k_B*T)).decompose()

<Quantity 7.06873635e+46 1 / s>